# Real-Time EMG to Angle Prediction with TensorFlow and LSL

This notebook demonstrates a more modular and robust approach to:
- Collecting real-time EMG (EXG) and angle (FingerPercentages) data from LSL streams.
- Applying online standardization to the EMG signals.
- Using a configurable CNN model to predict angles from recent EMG data.

**Features:**
- Adjustable EMG window size for the CNN input.
- More modular code structure with classes and separate functions.
- Graceful handling of missing or invalid data.
- Training on the most recent data available.

**Notes:**
- This example still assumes that LSL streams `filtered_exg` and `FingerPercentages` are available.
- Ensure you have the required dependencies installed: `pylsl`, `tensorflow`, `numpy`.
- Running this in real-time requires live LSL streams.

In [1]:
import time
import numpy as np
import threading
import tensorflow as tf
from pylsl import StreamInlet, resolve_stream, proc_ALL
from collections import deque

## Online Standardizer
This class maintains a running mean and std using Welford's algorithm, allowing for online standardization of incoming data.


In [2]:
class OnlineStandardizer:
    """
    Maintains a running mean and std using Welford's algorithm.
    """
    def __init__(self, epsilon=1e-7):
        self.count = 0
        self.mean = 0.0
        self.M2 = 0.0
        self.epsilon = epsilon
        
    def copy(self):
        new = OnlineStandardizer(self.epsilon)
        new.count = self.count
        new.mean = self.mean
        new.M2 = self.M2
        return new

    def update(self, x: np.ndarray):
        batch_count = x.shape[0]
        
        if batch_count == 0:
            return

        new_count = self.count + batch_count
        
        delta = x - self.mean
        self.mean += np.sum(delta, axis=0) / new_count
        delta2 = x - self.mean
        self.M2 += np.sum(delta * delta2, axis=0)
        self.count = new_count

    def get_stats(self):
        if self.count < 2:
            return self.mean, np.ones_like(self.mean)
        var = self.M2 / (self.count - 1)
        std = np.sqrt(var)
        std[std < self.epsilon] = self.epsilon
        return self.mean, std

    def apply(self, x: np.ndarray):
        mean, std = self.get_stats()
        x = (x - mean) / std
        return x

## EMG-Angle Collector
This class:
- Connects to specified LSL streams for EMG and angles.
- Continuously collects data in a background thread.
- Maintains a buffer of recent data.
- Rectifies EMG Signals and Applies Thresholds to Angles
- Provides a method to extract the most recent data window for training.

In [3]:
class NumpyBuffer:
    def __init__(self, capacity, shape, dtype):
        self.capacity = capacity
        self.shape = shape
        self.dtype = dtype
        self.buffer = np.zeros((capacity, *shape), dtype=dtype)
        self.timestamps = np.zeros(capacity, dtype=np.float64)
        self.size = 0
        
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        # Adjust to work with negative indices and slices
        def adjust_index(idx, limit=self.size):
            if idx < 0:
                idx += self.size
            if idx < 0 or idx >= limit:
                raise IndexError("Index out of bounds.")
            return idx
        
        if isinstance(idx, int):
            idx = adjust_index(idx)
            return self.timestamps[idx], self.buffer[idx]
        elif isinstance(idx, slice):
            start, stop, step = idx.indices(self.size)
            start = adjust_index(start)
            stop = adjust_index(stop, self.size + 1)
            timestamps = self.timestamps[start:stop:step]
            buffer = self.buffer[start:stop:step]
            return timestamps, buffer
        else:
            raise TypeError("Invalid index type.")
        
    def expand(self):
        new_buffer = np.zeros((self.capacity * 2, *self.shape), dtype=self.dtype)
        new_timestamps = np.zeros(self.capacity * 2, dtype=np.float64)
        new_buffer[:self.size] = self.buffer
        new_timestamps[:self.size] = self.timestamps
        self.buffer = new_buffer
        self.timestamps = new_timestamps
        self.capacity *= 2
    
    def extend(self, timestamps, samples):
        if len(timestamps) != len(samples):
            raise ValueError("Timestamps and samples must have the same length.")
        
        new_size = len(samples)
        
        if new_size == 0:
            return
        
        if self.size + new_size > self.capacity:
            self.expand()

        self.buffer[self.size:self.size+new_size] = samples
        self.timestamps[self.size:self.size+new_size] = timestamps
        self.size += new_size

    def get_numpy_buffers(self):
        return self.timestamps[:self.size], self.buffer[:self.size]

    def clear(self):
        self.size = 0

class EMGAngleCollector:
    def __init__(self, emg_stream_name="filtered_exg", angle_stream_name="FingerPercentages",
                 buffer_seconds=5, finger_thresholds = ((0.825, 0.9), (0.7, 0.875), (0.75, 0.875), (0.7, 0.8), (0.7, 0.825))):
        print("Resolving EMG (EXG) stream...")
        emg_streams = resolve_stream('name', emg_stream_name)
        if not emg_streams:
            raise RuntimeError(f"No EMG stream found with name '{emg_stream_name}'.")
        
        print("Resolving angle (MP) stream...")
        angle_streams = resolve_stream('name', angle_stream_name)
        if not angle_streams:
            raise RuntimeError(f"No angle stream found with name '{angle_stream_name}'.")

        self.emg_inlet = StreamInlet(emg_streams[0], processing_flags=proc_ALL, max_buflen=2)
        self.angle_inlet = StreamInlet(angle_streams[0], processing_flags=proc_ALL, max_buflen=2)
        self.finger_thresholds = np.array(finger_thresholds)

        # Get EMG stream info
        emg_info = self.emg_inlet.info()
        self.emg_rate = emg_info.nominal_srate()
        if self.emg_rate <= 0:
            raise ValueError("EMG stream sampling rate is not set or zero.")
        self.num_exg_channels = emg_info.channel_count()
        if self.num_exg_channels <= 0:
            raise ValueError("EMG stream channel_count is not set or zero.")

        # Get angle stream info
        angle_info = self.angle_inlet.info()
        self.angle_rate = angle_info.nominal_srate()
        if self.angle_rate <= 0:
            raise ValueError("Angle stream sampling rate is not set or zero.")
        self.num_mp_channels = angle_info.channel_count()
        if self.num_mp_channels <= 0:
            raise ValueError("Angle stream channel_count is not set or zero.")

        self.capacity = int(buffer_seconds * self.emg_rate)

        # Deques to store EMG and angle data
        self.emg_deque = deque(maxlen=self.capacity)
        self.emg_timestamp_deque = deque(maxlen=self.capacity)
        self.angle_deque = deque(maxlen=self.capacity)
        self.angle_timestamp_deque = deque(maxlen=self.capacity)
        
        # Standardizer for EMG data
        self.standardizer = OnlineStandardizer()

        self.data_lock = threading.Lock()
        self.stop_flag = False
        
        self.update_fps = 50
        self.update_interval = 1 / self.update_fps

        self.thread = threading.Thread(target=self.run, daemon=True)
        self.thread.start()

    def run(self):
        last_time = time.perf_counter()
        while not self.stop_flag:
            self.update()
            current_time = time.perf_counter()
            elapsed_time = current_time - last_time
            if elapsed_time < self.update_interval:
                time.sleep(self.update_interval - elapsed_time)
            last_time = current_time
    
    def get_emg_standardizer(self):
        with self.data_lock:
            return self.standardizer.copy()

    def update(self):
        angle_data, angle_timestamps = self.angle_inlet.pull_chunk()
        emg_data, emg_timestamps = self.emg_inlet.pull_chunk()

        if not emg_data or not angle_data:
            return

        # Convert to arrays
        emg_data = np.array(emg_data, dtype=np.float32)
        angle_data = np.array(angle_data, dtype=np.float32)
        
        # Now create nan_mask safely - For now we won't remove any data
        # emg_nan_mask = np.isnan(emg_data).any(axis=1)
        # angle_nan_mask = np.isnan(angle_data).any(axis=1)
        # emg_data = emg_data[~emg_nan_mask]
        # angle_data = angle_data[~angle_nan_mask]
        # emg_timestamps = np.array(emg_timestamps)[~emg_nan_mask]
        # angle_timestamps = np.array(angle_timestamps)[~angle_nan_mask]
        
        # Rectify EMG data
        emg_data = np.abs(emg_data)
        
        # Clip the angle data to the thresholds
        angle_data = np.clip(angle_data, self.finger_thresholds[:, 0], self.finger_thresholds[:, 1])
        angle_data = (angle_data - self.finger_thresholds[:, 0]) / (self.finger_thresholds[:, 1] - self.finger_thresholds[:, 0])
        
        with self.data_lock:
            if len(emg_data) != 0:
                self.emg_deque.extend(emg_data)
                self.emg_timestamp_deque.extend(emg_timestamps)
                self.standardizer.update(emg_data)
            if len(angle_data) != 0:
                self.angle_deque.extend(angle_data)
                self.angle_timestamp_deque.extend(angle_timestamps)
                
    def get_data(self):
        '''
        Returns the EMG and angle data in the deques as numpy
        arrays, and the standardizer object.
        '''
        with self.data_lock:
            emg = np.array(self.emg_deque)
            emg_timestamp = np.array(self.emg_timestamp_deque)
            angle = np.array(self.angle_deque)
            angle_timestamp = np.array(self.angle_timestamp_deque)
            self.emg_deque.clear()
            self.emg_timestamp_deque.clear()
            self.angle_deque.clear()
            self.angle_timestamp_deque.clear()
            standardizer = self.standardizer.copy()
        return (emg_timestamp, emg), (angle_timestamp, angle), standardizer
                
    def put_numpy_buffers(self, emg_buffer: NumpyBuffer, angle_buffer: NumpyBuffer):
        timestamped_emg, timestamped_angle = self.get_data()
        emg_buffer.extend(timestamped_emg[0], timestamped_emg[1])
        angle_buffer.extend(timestamped_angle[0], timestamped_angle[1])
    
    def stop(self):
        self.stop_flag = True
        self.thread.join()

## Data Generator

This generator yields batches of data to the model for training. It:
- Continuously calls `get_latest_batch` to get the most recent window.
- Checks for NaNs or Infs.
- Adds a batch dimension.


In [4]:
def data_generator(collector: EMGAngleCollector, batch_size:int, window_size: int, exg_buffer: NumpyBuffer, mp_buffer: NumpyBuffer):   
    while True:
        timestamped_exg, timestamped_mp, standardizer = collector.get_data()
        
        # Put the data in the buffers
        exg_buffer.extend(timestamped_exg[0], timestamped_exg[1])
        mp_buffer.extend(timestamped_mp[0], timestamped_mp[1])
        
        # Grab the earlier timestamp
        exg_timestamps, exg_data = exg_buffer.get_numpy_buffers()
        mp_timestamps, mp_data = mp_buffer.get_numpy_buffers()
        
        # Standardize the emg data
        exg_data = standardizer.apply(exg_data)
        
        mp_idx = -1
        exg_idx = -1
        
        # These 2 while loops are to ensure that the timestamps are aligned
        # With the mp data ahead of the exg data
        if len(mp_timestamps) == 0 or len(exg_timestamps) == 0:
            time.sleep(0.01)
            continue
        
        while mp_idx > -len(mp_timestamps) and mp_timestamps[mp_idx] > exg_timestamps[exg_idx]:
            mp_idx -= 1
        # mp data should be ahead of exg data
        while exg_idx > -len(exg_timestamps) and mp_timestamps[mp_idx] < exg_timestamps[exg_idx]:
            exg_idx -= 1
            
        # Ensure indices are within bounds
        if abs(mp_idx) > len(mp_timestamps) or abs(exg_idx) > len(exg_timestamps):
            time.sleep(0.01)
            continue
        
        # Create a windowed dataset
        X_batch = []
        y_batch = []
        for _ in range(batch_size):
            if (exg_idx - window_size) < -len(exg_data):
                break  # Not enough data for another sample
            
            # check for nan values in mp
            while mp_idx > -len(mp_data) and np.isnan(mp_data[mp_idx]).any():
                mp_idx -= 1
                exg_idx -= 1
            
            if mp_idx < -len(mp_data):
                break # Not enough data for another sample
            
            if (exg_idx - window_size) < -len(exg_data):
                break  # Not enough data for another sample
            
            # Get the window of data
            exg_window = exg_data[exg_idx - window_size:exg_idx]
            mp_window = mp_data[mp_idx]
            
            # Append to the batch
            X_batch.append(exg_window)
            y_batch.append(mp_window)
            
            # Move the index
            exg_idx -= 1
            mp_idx -= 1
            
        if len(X_batch) == 0:
            continue
        
        X_batch = np.array(X_batch, dtype=np.float32)  # (batch_size, window_size, num_exg_channels)
        y_batch = np.array(y_batch, dtype=np.float32)  # (batch_size, num_mp_channels)
        
        # # print the distribution of the target
        print('Mean', np.mean(y_batch, axis=0))
        # print('STD', np.std(y_batch, axis=0))

        yield X_batch, y_batch

## Model Creation

A simple CNN model for sequence data. The kernel size and number of filters can be adjusted.

We make the model creation more modular to easily tweak hyperparameters.


In [5]:
def create_model(sequence_length, num_exg_channels, num_mp_channels, filters=32, kernel_size=3):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(sequence_length, num_exg_channels)),
        tf.keras.layers.GaussianNoise(0.05),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.Conv1D(filters=filters*2, kernel_size=kernel_size, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.Conv1D(filters=filters*2, kernel_size=kernel_size, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        # Output one value per MP channel
        tf.keras.layers.Dense(num_mp_channels)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

## Training Loop

Here we:
- Initialize the collector with chosen parameters.
- Determine the sequence_length (window_size) from user input.
- Create a `tf.data.Dataset` from the data generator.
- Train the model.

In [6]:
# User-configurable parameters
EMG_STREAM_NAME = "filtered_exg"
ANGLE_STREAM_NAME = "FingerPercentages"
COLLECTOR_BUFFER_SECONDS = 5
EXPECTED_SESSION_SECONDS = 60 * 10
WINDOW_SIZE = 50  # Number of EMG samples per example
BATCH_SIZE = 1024 # Examples per training step
STEPS_PER_EPOCH = 10
EPOCHS = 1000
FILTERS = 32
KERNEL_SIZE = 3

# Initialize collector
collector = EMGAngleCollector(
    emg_stream_name=EMG_STREAM_NAME,
    angle_stream_name=ANGLE_STREAM_NAME,
    buffer_seconds=COLLECTOR_BUFFER_SECONDS,
)

# Initialize buffers
exg_buffer = NumpyBuffer(int(collector.emg_rate * EXPECTED_SESSION_SECONDS), (collector.num_exg_channels,), np.float32)
mp_buffer = NumpyBuffer(int(collector.angle_rate * EXPECTED_SESSION_SECONDS), (collector.num_mp_channels,), np.float32)

num_exg_channels = collector.num_exg_channels
num_mp_channels = collector.num_mp_channels

# Create dataset
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(collector, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE, exg_buffer=exg_buffer, mp_buffer=mp_buffer),
    output_signature=(
        tf.TensorSpec(shape=(BATCH_SIZE, WINDOW_SIZE, num_exg_channels), dtype=tf.float32),
        tf.TensorSpec(shape=(BATCH_SIZE, num_mp_channels), dtype=tf.float32)
    ),
    drop_remainder=True
).prefetch(1)

# Create model
model = create_model(WINDOW_SIZE, num_exg_channels, num_mp_channels, filters=FILTERS, kernel_size=KERNEL_SIZE)

time.sleep(5)

print("Starting training loop. Press Interrupt to stop.")
try:
    model.fit(dataset, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)
except KeyboardInterrupt:
    print("Training interrupted by user.")
finally:
    collector.stop()
    print("Stopped data collection and training.")

Resolving EMG (EXG) stream...
Resolving angle (MP) stream...


2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-12-13 02:16:38.947 (   0.049s) [          194FDC]      netinterfaces.cpp:91    I

Starting training loop. Press Interrupt to stop.
Epoch 1/1000
Mean [1. 1. 1. 1. 1.]
Mean [1. 1. 1. 1. 1.]
Stopped data collection and training.


2024-12-13 02:16:44.945596: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (912, 50, 16) where an element of shape (1024, 50, 16) was expected.
Traceback (most recent call last):

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (912, 50, 16) where an element of shape (1024, 50, 16) was expected.


2024-12-13 02:16:44.945657: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with sta

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (912, 50, 16) where an element of shape (1024, 50, 16) was expected.
Traceback (most recent call last):

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/sj/Dev/Python/robot-hand/.conda/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (912, 50, 16) 